# Sensor Replay Attack Example

In [ ]:
from IPython.display import display, HTML
display(HTML('<a target="_blank" href="https://colab.research.google.com/github/WaterFutures/EPyT-Flow/blob/main/docs/examples/sensor_replay_attack.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>'))

This example demonstrates how to add a sensor replay attack to a scenario -- more information can be found in the [documentation](https://epyt-flow.readthedocs.io/en/stable/tut.events.html#sensor-reading-attacks).

In [ ]:
%pip install epyt-flow --quiet

In [ ]:
from epyt_flow.data.benchmarks import load_leakdb_scenarios
from epyt_flow.simulation import ScenarioSimulator, SENSOR_TYPE_NODE_PRESSURE
from epyt_flow.simulation.events import SensorReplayAttack
from epyt_flow.utils import to_seconds, plot_timeseries_data

Load the first [LeakDB](https://epyt-flow.readthedocs.io/en/stable/epyt_flow.data.benchmarks.html#epyt_flow.data.benchmarks.leakdb.load_scenarios) Hanoi scenario:

In [ ]:
config = load_leakdb_scenarios(scenarios_id=["1"], use_net1=False, verbose=False)[0]

Create a new simulation:

In [ ]:
sim = ScenarioSimulator(scenario_config=config)

Set the simulation duration to two days:

In [ ]:
sim.set_general_parameters(simulation_duration=to_seconds(days=2))

Add a sensor replay attack -- pressure readings at node "13" between 5hrs and 7hrs after simulation start (time steps 10 - 15) are replaced by the historical readings collected from the first 150min (i.e. first 5 time steps).

This is done by creating an instance of the [SensorReplayAttack](https://epyt-flow.readthedocs.io/en/stable/epyt_flow.simulation.events.html#epyt_flow.simulation.events.sensor_reading_attack.SensorReplayAttack) class and adding it to the scenarion by calling [add_sensor_reading_event()](https://epyt-flow.readthedocs.io/en/stable/epyt_flow.simulation.html#epyt_flow.simulation.scenario_simulator.ScenarioSimulator.add_sensor_reading_event):

In [ ]:
sensor_replay_attack = SensorReplayAttack(replay_data_time_window_start=0,
                                          replay_data_time_window_end=to_seconds(minutes=150),
                                          start_time=to_seconds(hours=5),
                                          end_time=to_seconds(hours=7),
                                          sensor_id="13",
                                          sensor_type=SENSOR_TYPE_NODE_PRESSURE)

sim.add_sensor_reading_event(sensor_replay_attack)

Run the entire simulation:

In [ ]:
scada_data = sim.run_simulation()

Retrieve and plot the pressure readings:

In [ ]:
pressure_readings = scada_data.get_data_pressures(sensor_locations=["13"])

In [ ]:
scada_data.plot_pressures(sensor_locations=["13"])

Readings for time 10 - 15 are same as the first 5 readings!

In [ ]:
plot_timeseries_data(pressure_readings[:20].T,    # The same as the first 5 readings!
                     x_axis_label="Time (30min steps)",
                     y_axis_label="Pressure in $m$")

Sensor readings WITHOUT the sensor replay attack

In [ ]:
# Remove sensor replay attack
scada_data.sensor_reading_events = []

# Recompute and show final sensor readings
pressure_readings = scada_data.get_data_pressures(sensor_locations=["13"])
plot_timeseries_data(pressure_readings[:20].T,
                     x_axis_label="Time (30min steps)",
                     y_axis_label="Pressure in $m$")

Do not forget to close the simulation!

In [ ]:
sim.close()